In [1]:
import os

In [2]:
# normal imgs
fedora_path = "/home/sisa/sat-data/MODIS_MOD09GA_GERITAFRA_31day_DEC2023/"
extern_path = "/mnt/f/Bachelorarbeit/sat-data/modis/MODIS_MOD09GA_GERITAFRA_31day_DEC2023/MOD_RGB_corrected_15percent/"

# predictions
extern_prediction_path = "/mnt/f/Bachelorarbeit/sat-data/modis/MODIS_DEC2023_Predictions_Binary/"


# experiment 2 path
extern_MODIS_path =  "/mnt/f/Bachelorarbeit/sat-data/modis/MODIS_MOD09GA_GERITAFRA_31day_DEC2023/"


modis_tower_path = "/home/mxh/MODIS_MOD09GA_GERITAFRA_31day_DEC2023/"
modis_volume_path="/media/mxh/Volume/SAT-DATA/MODIS_MOD09GA_GERITAFRA_31day_DEC2023/"

modis_win_path = "F:\\Bachelorarbeit\\sat-data\\modis\\MODIS_MOD09GA_GERITAFRA_31day_DEC2023\\"
modis_x_path = "/mnt/f/Bachelorarbeit/sat-data/modis/MODIS_MOD09GA_GERITAFRA_31day_DEC2023/"
modis_wsl_path = "/home/sisa/sat-data/MODIS_MOD09GA_GERITAFRA_31day_DEC2023/"# "\\wsl.localhost\fedora39\home\sisa\sat-data"

# experiments
modis_exp2_wsl_path = "/home/sisa/sat-data/MODIS_MOD09GA_GERITAFRA_31day_DEC2023/experiment2/"


path = modis_tower_path

In [3]:
files = os.listdir(path)#
files

['MOD09GA.A2023342.h18v04.061.2023344024915_solar_azimuth_2400_wgs84_warped.tmp.tiff',
 'MOD09GA.A2023356.h18v04.061.2023361000333_cloud_state_mask.svg',
 'MOD09GA.A2023356.h18v04.061.2023361000333_rgb_scaled_tc_refl_0.15_480.svg',
 'MOD09GA.A2023354.h18v04.061.2023356042206.hdf',
 'MOD09GA.A2023344.h18v04.061.2023346030519_sensor_azimuth_2400_wgs84_warped.tmp.tiff',
 'MOD09GA.A2023341.h18v04.061.2023343024753_cloud_state_mask.png',
 'MOD09GA.A2023350.h18v04.061.2023352030330_merged_2400_Unet-v2-binary_240526173722_prediction_int32_mask.tiff',
 'MOD09GA.A2023339.h18v04.061.2023341041457_solar_azimuth_2400_wgs84_warped.tmp.tiff',
 'MOD09GA.A2023335.h18v04.061.2023340033530_solar_zenith_500_wgs84_warped.tmp.tiff',
 'MOD09GA.A2023336.h18v04.061.2023338023416_upscaled_angles_2400.tiff',
 'MOD09GA.A2023365.h18v04.061.2024004181336_sensor_azimuth_2400_wgs84_warped.tiff',
 'MOD09GA.A2023359.h18v04.061.2023361050233_cirrus_mask',
 'MOD09GA.A2023351.h18v04.061.2023353030525_cloud_state_mask.png

In [4]:
hdf_files = sorted([f for f in files if f[-4:] == ".hdf"])


## Definitionen

In [5]:
def generate_ql_modis_commands(folder="../MODIS_DEC2023_Predictions_Binary", filename="", suffix="", extension=".tiff", date=1):
    """
    filename like   MOD09GA.A2023335.h18v04.061.2023340033530
    suffix like     _merged_2400_binary_M1_x_prediction_bin_mask
    extensions like .tiff
    date start counting at 1!
    """
    ql_modis_template = f"--ql-modis=(--file {folder}{filename}{suffix}{extension} --date {date})"
    return ql_modis_template


In [6]:
def generate_ql_img_commands(folder="../MODIS_DEC2023_Predictions_Binary/", filename="", suffix="", extension=".tiff", date=1):
    """
    filename like   MOD09GA.A2023335.h18v04.061.2023340033530
    suffix like     _merged_2400_binary_M1_x_prediction_bin_mask
    extensions like .tiff
    date start counting at 1!
    """
    ql_img_template = f"--ql-img=(--file {folder}{filename}{suffix}{extension} --date {date} --interp-ranges=[1,1] --non-interp-ranges=[0,0])"
    return ql_img_template

In [7]:
def generate_modis_hdf_commands(folder="../", filename="", layers=(10,11,12,13,14,15,16), date=1, extension=".hdf", suffix=""):
    """
    filename like   MOD09GA.A2023335.h18v04.061.2023340033530
    extensions like .hdf
    date start counting at 1!

    layers = (10, 11,12,13,14,15,16)
    """
    layer_string = "("
    for layer in layers:
        layer_string += str(layer) + " "
    layer_string += ")"

    modis_hdf_template = f"--img=(--file {folder}{filename}{suffix}{extension} -l {layer_string} --date {date})"
    return modis_hdf_template

## Generierung

In [8]:
# definieren der tage
start = 6
stop  = 19

### interp

In [11]:
modis_hdf_commands=[]
ql_modis_commands=[]
original_modis_ql_commands=[]
ql_img_commands = []
modis_tiff_commands=[]

model_suffix= "_merged_2400_binary_M1_240605215803_prediction_bin_mask" #  "_merged_2400_Unet-v2-binary_240526173722_prediction_bin_mask"
original_ql_suffix = "_state_1km_1_2400"
tiff_suffix="_merged_2400"


for i, file in enumerate(hdf_files[start:stop]):
    # remember, i has to be ++ed for imfu
    #print(i+1, file)
    filename = file[:-4]
    extension = ".hdf"
    date = i+1

    modis_hdf_command=generate_modis_hdf_commands(filename=filename, date=date, extension=extension)
    modis_hdf_commands.append(modis_hdf_command)

    #(0,1,2,3,4,5,6)
    modis_tiff_command=generate_modis_hdf_commands(folder="", filename=filename, extension=".tiff", layers=(0,1,2,3,4,5,6), date=date, suffix=tiff_suffix)
    modis_tiff_commands.append(modis_tiff_command)
    # quality layers
    ql_modis_command = generate_ql_modis_commands(filename=filename, suffix=model_suffix, date=date)
    ql_modis_commands.append(ql_modis_command)

    original_ql_modis_command = generate_ql_modis_commands(folder="../",filename=filename, suffix=original_ql_suffix, date=date)
    original_modis_ql_commands.append(original_ql_modis_command)

    ql_img_command = generate_ql_img_commands(folder="",filename=filename, suffix=model_suffix, date=date)
    ql_img_commands.append(ql_img_command)

In [12]:
# print commands you want listed
for c in ql_img_commands:
    print(c)

--ql-img=(--file MOD09GA.A2023341.h18v04.061.2023343024753_merged_2400_binary_M1_240605215803_prediction_bin_mask.tiff --date 1 --interp-ranges=[1,1] --non-interp-ranges=[0,0])
--ql-img=(--file MOD09GA.A2023342.h18v04.061.2023344024915_merged_2400_binary_M1_240605215803_prediction_bin_mask.tiff --date 2 --interp-ranges=[1,1] --non-interp-ranges=[0,0])
--ql-img=(--file MOD09GA.A2023343.h18v04.061.2023345024918_merged_2400_binary_M1_240605215803_prediction_bin_mask.tiff --date 3 --interp-ranges=[1,1] --non-interp-ranges=[0,0])
--ql-img=(--file MOD09GA.A2023344.h18v04.061.2023346030519_merged_2400_binary_M1_240605215803_prediction_bin_mask.tiff --date 4 --interp-ranges=[1,1] --non-interp-ranges=[0,0])
--ql-img=(--file MOD09GA.A2023345.h18v04.061.2023347025532_merged_2400_binary_M1_240605215803_prediction_bin_mask.tiff --date 5 --interp-ranges=[1,1] --non-interp-ranges=[0,0])
--ql-img=(--file MOD09GA.A2023346.h18v04.061.2023348030312_merged_2400_binary_M1_240605215803_prediction_bin_mask.t

### imgcompare